https://chromedriver.chromium.org/downloads

In [ ]:
#第一次run
%pip install ipywidgets
%pip install IPython
%pip install selenium

In [1]:
import ipywidgets as widgets
from IPython.display import display
import os, re, shutil

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

import pyperclip

from xml.dom import minidom

# driver = webdriver.Chrome()
# driver = webdriver.Firefox()
url = "https://yqnn.github.io/svg-path-editor/"

# driver.get(url)

In [13]:

BASE_PATH = '/Users/willchan/Desktop/HTW/missingStroke/toDo20230921/3forTest/'
TARGET_PATH = '/Users/willchan/Desktop/HTW/missingStroke/toDo20230921/' + 'TO_GAEL/'
base_path = BASE_PATH
file_no = ""
org_file = []
stroke_no = 0
last_stroke = ""
cur_stroke = ""
next_stroke = ""
paste_flag = False
# input_order = widgets.Text()
# display(input_order)

pre = f"{file_no}_"

# new_folder_location = "/Users/willchan/Desktop" #想擺邊
# new_order_folder = '/Users/willchan/Desktop/batch_1_regen_shape/shape_fixed/'

try:
    os.mkdir(TARGET_PATH)
except:
    pass

def extend(input_list: str):
    split_string = input_list.split()
    temp_list = []

    for substring in split_string:
        if '-' in substring:
            substring_split = substring.split('-')
            start = int(substring_split[0])
            end = int(substring_split[1])
            temp_list.extend(range(start, end+1))
        else:
            temp_list.append(int(substring))
    
    return temp_list  


def copy_stroke_svg(s):
    # global file_no, base_path, org_file, stroke_no
    base_path = BASE_PATH
    file_no = ""
    org_file = []
    stroke_no = 0

    # print(s)
    temp = s.split()
    
    file_no = temp[0]
    stroke_no = int(temp[1])
    
    base_path += file_no
    # print(base_path)
    for root, dirs, files in os.walk(base_path):
        for file in files:
            if file == ".DS_Store" or 'trace' in file or 'turn' in file:
                continue
            org_file.append(file)
    org_file = sorted(org_file, key=lambda x:int(x.split("_")[1]))
    # print(org_file)
   
    pyperclip.copy(get_svg_path(org_file[stroke_no]))
    
def reset():
    global new_folder_location, base_path, file_no, org_file, pre
    new_order_folder = TARGET_PATH

    base_path = BASE_PATH
    file_no = ""
    org_file = []
    pre = f"{file_no}_"

    input_folder_number.value = ""
    input_duplicate_target_stroke.value = ""

    

def fix_svg(sender):
    global file_no, base_path, org_file, stroke_no, TARGET_PATH
    org_file = []
    # print(sender.value)


    # temp = sender.value.split()
    
    # file_no = temp[0]
    # stroke_no = int(temp[1])
    stroke_no = int(sender.value)
    
    base_path = os.path.join(TARGET_PATH,file_no) 
    print('base path:',base_path)
    for root, dirs, files in os.walk(base_path):
        for file in files:
            if file == ".DS_Store" or 'trace' in file or 'turn' in file:
                continue
            org_file.append(file)
    org_file = sorted(org_file, key=lambda x:int(x.split("_")[1]))
    # print(org_file)

    send_svg(get_svg_path(org_file[stroke_no]))
    # reset()

def get_file_number(sender) -> int:
    global file_no
    
    file_no = sender.value
    print('file no:', file_no)

def new_order(sender):
    global new_order_folder
    # order = sender.value.split(" ")
    order = extend(sender.value)
    # print("order" , order) 
    l = len(order)
    if l != len(org_file):
        raise Exception("Length Not match")
    
    # for i in range(l):
    #     if not order[i].isdigit():
    #         raise Exception(f"{order[i]} not a digit")
    #     if str(i) not in order:
    #         # print(i)
    #         raise Exception(f"{i} not in the list")

    os.mkdir(os.path.join(new_order_folder, file_no))

    for i in range(len(org_file)):
        # if order[i] == '-':
        #     continue
        suffix = (re.search(r'_[A-Z](.*)', org_file[i])).group()
        org_path = f'{base_path}/{org_file[int(order[i])]}'
        # print(order[i])
        target_path = f'{new_order_folder}/{file_no}/{file_no}_{i}{suffix}' #TODO

        # print(org_path)
        shutil.copyfile(org_path, target_path)
        
    # print("done")
    reset()

def duplicate_stroke(sender):
    global file_no, TARGET_PATH, base_path, org_file

    #strokeNo, orgStrokeName, NewStrokeName
    temp = sender.value.split()
    strokeNo = temp[0]
    if len(temp) > 1:
        orgStrokeName = temp[1].upper()
        NewStrokeName = temp[2].upper()

    base_path += file_no
    for root, dirs, files in os.walk(base_path):
        for file in files:
            if file == ".DS_Store" or 'trace' in file or 'turn' in file:
                continue
            org_file.append(file)
    org_file = sorted(org_file, key=lambda x:int(x.split("_")[1]))
    target_path = os.path.join(TARGET_PATH, file_no)
    os.mkdir(target_path)
    strokeNo = len(org_file) if strokeNo == '-1' else strokeNo

    for i in range(int(strokeNo)):
        shutil.copyfile(os.path.join(base_path, org_file[i]), os.path.join(target_path, org_file[i]))

    shutil.copyfile(os.path.join(base_path, f'{file_no}_trace.svg'), os.path.join(target_path, f'{file_no}_trace.svg'))
    shutil.copyfile(os.path.join(base_path, f'{file_no}_turn.svg'), os.path.join(target_path, f'{file_no}_turn.svg'))

    if strokeNo == len(org_file):
        return

    shutil.copyfile(os.path.join(base_path, org_file[int(strokeNo)]), os.path.join(target_path, f'{file_no}_{int(strokeNo)}_{orgStrokeName}.svg'))
    shutil.copyfile(os.path.join(base_path, org_file[int(strokeNo)]), os.path.join(target_path, f'{file_no}_{int(strokeNo) + 1}_{NewStrokeName}.svg'))

    for i in range(int(strokeNo)+1, len(org_file)):
        suffix = (re.search(r'_[A-Z](.*)', org_file[i])).group()
        shutil.copyfile(os.path.join(base_path, org_file[i]), os.path.join(target_path, f'{file_no}_{i + 1}{suffix}'))



    print('done')


def wait_send(by, path, send):
    WebDriverWait(driver, 180).until(EC.presence_of_element_located((by, path)))
    driver.find_element(by, path).send_keys(send)

def send_svg(svg: str):
    WebDriverWait(driver, 180).until(EC.presence_of_element_located((By.CSS_SELECTOR,"textarea")))
    driver.find_element(By.CSS_SELECTOR,"textarea").clear()
    wait_send(By.CSS_SELECTOR,"textarea", svg)
    # driver.execute_script("document.querySelector('.drawings.ng-tns-c94-0').style.transform='rotateX(180deg)'")

def get_svg_path(file):
    path = f"{base_path}/{file}"

    doc = minidom.parse(path)  # parseString also exists
    # print(doc.getElementsByTagName("ns0:path"))
    path_strings = [path.getAttribute('d') for path
                    in doc.getElementsByTagName('ns0:path')]
    doc.unlink()
    return (re.sub(r'([A-Y])', r'\1 ', re.sub(r'(\d{3})(Q\d{3})', r'\1 \2' ,path_strings[0])))

def copy_stroke_svg(s):
    # global file_no, base_path, org_file, stroke_no
    # reset()
    base_path = BASE_PATH
    file_no = ""
    org_file = []
    stroke_no = 0

    print(s)
    temp = s.split()
    
    file_no = temp[0]
    
    base_path += file_no
    print(base_path)
    for root, dirs, files in os.walk(base_path):
        for file in files:
            if file == ".DS_Store" or 'trace' in file or 'turn' in file:
                continue
            org_file.append(file)
    org_file = sorted(org_file, key=lambda x:int(x.split("_")[1]))
    print(org_file)
    stroke_no = [x for x in range(len(org_file))] if len(temp)== 1 else extend(temp[1])
    
    abc = ''

    for no in stroke_no:
        abc += get_svg_path(org_file[no])
    
    pyperclip.copy(abc)

# input_folder.on_submit(fix_svg)

# input_order.on_submit(new_order)



<h1><strong>複製筆畫</strong></h1>

In [15]:
#第一格 file number e.g. 39489
#第二格 複製邊一筆 e.g. 14 s hzg

input_folder_number = widgets.Text()
input_duplicate_target_stroke = widgets.Text()
display(input_folder_number)
display(input_duplicate_target_stroke)

input_folder_number.on_submit(get_file_number)
input_duplicate_target_stroke.on_submit(duplicate_stroke)

Text(value='')

Text(value='')

/var/folders/y7/ytwy5wl55bs956vg5hlz1n1r0000gn/T/ipykernel_20139/2179293542.py:9: DeprecationWarning: on_submit is deprecated. Instead, set the .continuous_update attribute to False and observe the value changing with: mywidget.observe(callback, 'value').
  input_folder_number.on_submit(get_file_number)
/var/folders/y7/ytwy5wl55bs956vg5hlz1n1r0000gn/T/ipykernel_20139/2179293542.py:10: DeprecationWarning: on_submit is deprecated. Instead, set the .continuous_update attribute to False and observe the value changing with: mywidget.observe(callback, 'value').
  input_duplicate_target_stroke.on_submit(duplicate_stroke)


file no: 39425


<h1><strong>改SVG</strong></h1>

In [ ]:
#開browser

driver = webdriver.Chrome()
# driver = webdriver.Firefox()
url = "https://yqnn.github.io/svg-path-editor/"
driver.get(url)


In [ ]:
#改邊筆svg

input_svg_stroke_no = widgets.Text()
display(input_svg_stroke_no)
input_svg_stroke_no.on_submit(fix_svg)


In [40]:
#改完run for saving
def change_svg_path(file):

    changed_path = driver.find_element(By.CSS_SELECTOR,"textarea").get_attribute('value')

    path = f"{base_path}/{file}"

    # doc = minidom.parse(path) 
    # # print(doc.getElementsByTagName("ns0:path"))
    # path_strings = [path.getAttribute('d') for path
    #                 in doc.getElementsByTagName('ns0:path')]
    # doc.unlink()

    doc = minidom.parse(path)

    nodes = doc.getElementsByTagName('ns0:path')

    for node in nodes:
        # 修改節點的屬性
        node.setAttribute('d', changed_path)

        # 取得節點的屬性值
        # attribute_value = node.getAttribute('d')

    with open(path, 'w') as f:
        doc.writexml(f)
    # temp_new_folder = new_order_folder + file_no #TODO
    # if os.path.isdir(temp_new_folder):
    #     shutil.rmtree(temp_new_folder)
    # shutil.copytree(base_path ,temp_new_folder)

    reset()
    print("done")

change_svg_path(org_file[stroke_no])

done


In [45]:
def get_specify_stroke_svg(s):
    global TARGET_PATH
    # reset()
    base_path = TARGET_PATH #本身個folder係邊
    file_no = ""
    org_file = []
    stroke_no = 0

    print(s)
    temp = s.split()
    
    file_no = temp[0]
    
    base_path += file_no
    print(base_path)
    for root, dirs, files in os.walk(base_path):
        for file in files:
            if file == ".DS_Store" or 'trace' in file or 'turn' in file:
                continue
            org_file.append(file)
    org_file = sorted(org_file, key=lambda x:int(x.split("_")[1]))
    print(org_file)
    stroke_no = [x for x in range(len(org_file))] if len(temp)== 1 else extend(temp[1])
    
    abc = ''

    for no in stroke_no:
        abc += get_svg_path(org_file[no])
    
    pyperclip.copy(abc)

In [47]:
base_path

'/Users/willchan/Desktop/HTW/missingStroke/toDo20230921/3forTest/'

In [46]:
get_specify_stroke_svg('39489 12')

39489 12
/Users/willchan/Desktop/HTW/missingStroke/toDo20230921/TO_GAEL/39489
['39489_0_H.svg', '39489_1_S.svg', '39489_2_H.svg', '39489_3_H.svg', '39489_4_HZG.svg', '39489_5_P.svg', '39489_6_P.svg', '39489_7_H.svg', '39489_8_P.svg', '39489_9_N.svg', '39489_10_H.svg', '39489_11_H.svg', '39489_12_H.svg', '39489_13_S.svg', '39489_14_ZWG.svg', '39489_15_S.svg', '39489_16_D.svg', '39489_17_D.svg', '39489_18_D.svg', '39489_19_D.svg']


FileNotFoundError: [Errno 2] No such file or directory: '/Users/willchan/Desktop/HTW/missingStroke/toDo20230921/3forTest//39489_12_H.svg'